## Machine learning
Questo notebook sarà dedicato alla parte di regressione e classificazione

Iniziamo con la prima task, ovvero prevedere l'attività di twitter a livello provinciale, iniziamo discretizzando il tempo in bins di 30 minuti. Questo è dovuto al fatto che, grazie all'EDA, abbiamo osservato dei picchi di traffico specialmente nelle ore serali (da qui la necessità di avere una risoluzione temporale abbastanza fine) ma allo stesso tempo avendo 27k records in 62 giorni non volevamo avere dei binning con troppa poca popolazione. Da qui la nostra scelta.


In [75]:
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path
import numpy
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from functions import *
import seaborn as sns
from numpy import linspace
import json
from shapely.geometry import Point

# funzioni di sk-learn
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.metrics import matthews_corrcoef, accuracy_score, confusion_matrix, plot_confusion_matrix
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split



#custom lib
import make_dataset as m_d

In [5]:
# ML regressors and classificators
from sklearn.linear_model import LogisticRegression

In [6]:
# devo prima unire i due dataset di line set -> nov e poi nov -> dicembre

#Problema: il primo dato vien preso come dizionario
#Converto colonne
temp = m_d.safe_import('SET-2')
temp=temp.rename(columns={'DG1000420': 'LINESET', "2013-12-01 00:00": 'Timestamp', "36.719997" : 'Value Amp'})
#Riaggiungo il primo dato
electro=appforth(temp,['DG1000420','2013-12-01 00:00',36.719997])

temp1 = m_d.safe_import('SET-2')
temp1=temp1.rename(columns={'DG1000420': 'LINESET', "2013-11-01 00:00": 'Timestamp', "37.439999" : 'Value Amp'})
#Riaggiungo il primo dato
electro_1=appforth(temp1,['DG1000420','2013-12-01 00:00',36.719997])

electro.append(electro_1)

lines = m_d.safe_import('SET-lines')
electro = electro.merge(right=lines, how='outer')


#Ci sono incompatibilità tra i record salvati quindi li droppo cattivo
electro = electro.rename(columns={'SQUAREID': 'cellId'})
electro = electro.dropna()


Index(['DG1000420', '2013-12-01 00:00', '36.719997'], dtype='object')
Index(['DG1000420', '2013-12-01 00:00', '36.719997'], dtype='object')
Index(['SQUAREID', 'LINESET', 'NR_UBICAZIONI'], dtype='object')


In [7]:
electro

,LINESET,Timestamp,Value Amp,cellId,NR_UBICAZIONI
0,DG1000420,2013-12-01 00:00,36.719997,4037,2
1,DG1000420,2013-12-01 00:00,36.719997,4154,13
2,DG1000420,2013-12-01 00:00,36.719997,4155,20
3,DG1000420,2013-12-01 00:00,36.719997,4156,2
4,DG1000420,2013-12-01 00:00,36.719997,4269,1
...,...,...,...,...,...
11298379,DG1056626,2013-12-31 23:50,75.775002,8502,1
11298380,DG1056626,2013-12-31 23:50,75.775002,8503,4
11298381,DG1056626,2013-12-31 23:50,75.775002,8621,1
11298382,DG1056626,2013-12-31 23:50,75.775002,8972,25


In [17]:
#data_path = Path('./data/raw')   #Per Cookiecutter
data_path = Path('./data/raw')
# subdivide the time series

#########QUESTE VAN SPOSTATE IN UN FILE
def date_divider(df):
    """
    #This Fuction extracts the data from an undivided data format YYYY-MM-DDThh-mm-ss
    #Where T stands for Time and has no further meaning
    """
    column_names = ["month", "day", "hours", "temperature", "rain"]
    out = pd.DataFrame(columns=column_names)
    N = len(df["created"])  # Numero tweets creati in un giorno
    for i in range(0, N):
        # Inizializzo ogni riga o crasha
        out.loc[i] = "NaN"
        # Tempo
        # Tw_final.loc[i]["year"]=int(tweets.loc[i]["created"][0:4])  #Chemmifrega dell'anno, tutti uguali
        out.loc[i]["month"] = int(df.loc[i]["created"][5:7])
        out.loc[i]["day"] = int(df.loc[i]["created"][8:10])
        out.loc[i]["hours"] = int(df.loc[i]["created"][11:13]) + 0.5 * (
            int(df.loc[i]["created"][14:16]) >= 30)
    return out



def Access_counter(df):
    """
    #Please note that the data must be preprocessed by "date_divider" in order to work
    #This function DOES NOT work with raw data
    #This function assign a label to the minutes of the raw data in order to match them to the time resolution of the temperature aquisition.
    #The output of this function is a dictionary containing all the days!!
    """
    # I create a Support dictionary in order to get all the accesses subdivided by day
    dict = {}
    # Lunghezza is the number of elements
    lunghezza = 0
    k = 0
    mesi = df["month"].unique()
    for mese in mesi:
        temp = df[df["month"] == mese]
        lunghezza = len(temp['day'].unique())
        for j in range(lunghezza):
            data = temp[temp["day"] == j]
            bins = numpy.linspace(0, 24, 49)
            dict[k], edges = numpy.histogram(data['hours'], bins)
            k = k+1
    return dict


In [18]:
################### Importo le cose che servono, vanno cancellate in fase CC #################
df = pd.read_csv("twitter_final.csv")
df 

,month,day,hours,temperature,rain,municipal,geometry
0,11,1,0.0,13.2,0.0,Trento,POINT (11.13 46.07)
1,11,1,0.0,8.4,0.0,Pinzolo,POINT (10.83 46.23)
2,11,1,0.5,8.8,0.0,Cavalese,POINT (11.46 46.29)
3,11,1,0.5,13.2,0.0,Rovereto,POINT (11.04 45.89)
4,11,1,0.5,11.8,0.0,San Michele all'Adige,POINT (11.12 46.2)
...,...,...,...,...,...,...,...
27936,12,31,0.0,-7.7,0.0,Vigo di Fassa,POINT (11.68 46.42)
27937,12,29,13.0,3.4,0.0,Predazzo,POINT (11.6 46.31)
27938,11,19,0.5,5.0,0.0,Sant'Orsola Terme,POINT (11.3 46.11)
27939,11,23,9.5,0.7,0.0,Sant'Orsola Terme,POINT (11.3 46.11)


In [19]:
# grid di riferimento, serve un ponte tra l'ID della cella e la posizione geografica
grid=m_d.safe_import("grid")

grid.merge(right=electro, how = 'inner', on = 'cellId')
#grid.merge(right=electro, how = 'inner', on = 'geometry')

#df = df.merge(right=electro, how = 'inner', on = )

Index(['cellId', 'geometry'], dtype='object')


,cellId,geometry,LINESET,Timestamp,Value Amp,NR_UBICAZIONI
0,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079...",DG1031221,2013-12-01 00:00,34.439999,4
1,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079...",DG1031221,2013-12-01 00:10,34.799999,4
2,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079...",DG1031221,2013-12-01 00:20,35.639999,4
3,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079...",DG1031221,2013-12-01 00:30,32.520000,4
4,155,"POLYGON ((10.91493 45.69100, 10.92777 45.69079...",DG1031221,2013-12-01 00:40,33.119999,4
...,...,...,...,...,...,...
11298379,11099,"POLYGON ((11.76477 46.51064, 11.77778 46.51033...",DG1013503,2013-12-31 23:10,12.825000,3
11298380,11099,"POLYGON ((11.76477 46.51064, 11.77778 46.51033...",DG1013503,2013-12-31 23:20,12.225000,3
11298381,11099,"POLYGON ((11.76477 46.51064, 11.77778 46.51033...",DG1013503,2013-12-31 23:30,12.100000,3
11298382,11099,"POLYGON ((11.76477 46.51064, 11.77778 46.51033...",DG1013503,2013-12-31 23:40,11.475000,3


In [20]:
# creo il target, ovvero la divisione per fascia oraria di quanto ci è stato dato
counts_x_day = Access_counter(df)
print(counts_x_day[12])

[ 5  3  6  0  0  0  0  3  7  0  0  0  0 13 27  2  0  7  2  2 12  2  6  1
  3  0  2  0  4  2 23 21  7  4  5 14 35 19 11  2  4  6 11 16  8  4  3  1]


## Weather

In [76]:
weather_json = json.load( open(m_d.data_path / m_d.files['weather'][0]) )
weather = gpd.GeoDataFrame(weather_json['features'])


#Elimino le colonne del vento (dati molto incompleti)
weather.drop(weather.columns[list(range(202,298))], axis=1, inplace=True)
weather.drop(columns=['minWind', "maxWind"], inplace=True)

#Svolgiamo infine i punti geometrici
weather['geometry'] = weather['geomPoint.geom'].apply(lambda x:Point(x['coordinates'][0], x['coordinates'][1]))
weather.drop(columns=['geomPoint.geom'],inplace=True)

pd.set_option('display.max_columns', None)
weather.head(5) 

,station,elevation,date,timestamp,minTemperature,maxTemperature,precipitation,temperatures.0000,temperatures.0015,temperatures.0030,temperatures.0045,temperatures.0100,temperatures.0115,temperatures.0130,temperatures.0145,temperatures.0200,temperatures.0215,temperatures.0230,temperatures.0245,temperatures.0300,temperatures.0315,temperatures.0330,temperatures.0345,temperatures.0400,temperatures.0415,temperatures.0430,temperatures.0445,temperatures.0500,temperatures.0515,temperatures.0530,temperatures.0545,temperatures.0600,temperatures.0615,temperatures.0630,temperatures.0645,temperatures.0700,temperatures.0715,temperatures.0730,temperatures.0745,temperatures.0800,temperatures.0815,temperatures.0830,temperatures.0845,temperatures.0900,temperatures.0915,temperatures.0930,temperatures.0945,temperatures.1000,temperatures.1015,temperatures.1030,temperatures.1045,temperatures.1100,temperatures.1115,temperatures.1130,temperatures.1145,temperatures.1200,temperatures.1215,temperatures.1230,temperatures.1245,temperatures.1300,temperatures.1315,temperatures.1330,temperatures.1345,temperatures.1400,temperatures.1415,temperatures.1430,temperatures.1445,temperatures.1500,temperatures.1515,temperatures.1530,temperatures.1545,temperatures.1600,temperatures.1615,temperatures.1630,temperatures.1645,temperatures.1700,temperatures.1715,temperatures.1730,temperatures.1745,temperatures.1800,temperatures.1815,temperatures.1830,temperatures.1845,temperatures.1900,temperatures.1915,temperatures.1930,temperatures.1945,temperatures.2000,temperatures.2015,temperatures.2030,temperatures.2045,temperatures.2100,temperatures.2115,temperatures.2130,temperatures.2145,temperatures.2200,temperatures.2215,temperatures.2230,temperatures.2245,temperatures.2300,temperatures.2315,temperatures.2330,temperatures.2345,precipitations.0000,precipitations.0015,precipitations.0030,precipitations.0045,precipitations.0100,precipitations.0115,precipitations.0130,precipitations.0145,precipitations.0200,precipitations.0215,precipitations.0230,precipitations.0245,precipitations.0300,precipitations.0315,precipitations.0330,precipitations.0345,precipitations.0400,precipitations.0415,precipitations.0430,precipitations.0445,precipitations.0500,precipitations.0515,precipitations.0530,precipitations.0545,precipitations.0600,precipitations.0615,precipitations.0630,precipitations.0645,precipitations.0700,precipitations.0715,precipitations.0730,precipitations.0745,precipitations.0800,precipitations.0815,precipitations.0830,precipitations.0845,precipitations.0900,precipitations.0915,precipitations.0930,precipitations.0945,precipitations.1000,precipitations.1015,precipitations.1030,precipitations.1045,precipitations.1100,precipitations.1115,precipitations.1130,precipitations.1145,precipitations.1200,precipitations.1215,precipitations.1230,precipitations.1245,precipitations.1300,precipitations.1315,precipitations.1330,precipitations.1345,precipitations.1400,precipitations.1415,precipitations.1430,precipitations.1445,precipitations.1500,precipitations.1515,precipitations.1530,precipitations.1545,precipitations.1600,precipitations.1615,precipitations.1630,precipitations.1645,precipitations.1700,precipitations.1715,precipitations.1730,precipitations.1745,precipitations.1800,precipitations.1815,precipitations.1830,precipitations.1845,precipitations.1900,precipitations.1915,precipitations.1930,precipitations.1945,precipitations.2000,precipitations.2015,precipitations.2030,precipitations.2045,precipitations.2100,precipitations.2115,precipitations.2130,precipitations.2145,precipitations.2200,precipitations.2215,precipitations.2230,precipitations.2245,precipitations.2300,precipitations.2315,precipitations.2330,precipitations.2345,geometry
0,T0071,905,2013-11-01,1383260400,4.5,12.3,False,8.4,8.3,8.2,8.1,8.2,8.2,8.3,8.1,8.0,7.7,7.5,7.6,7.8,7.8,7.5,7.0,6.6,6.1,6.3,6.1,5.7,5.7,5.0,4.9,4.9,4.7,4.9,5.0,5.0,4.5,4.8,4.9,4.6,5.1,5.0,5.1,5.5,5.4,5.9,7.0,7.5,7.7,8.6,9.1,9.8,11.2,11.6,11.5,11.2,11.5,11.6,11.8,1

# Prepare the data for the ML dataset
Task 1: voglio cercare di predire il numero di tweets del giorno i-esimo a partire da alcuni parametri dei giorni i-1 e i-2. \
I parametri scelti sono: numero di tweets, temperatura, precipitazioni.... (ALTRO?) \
Si noti che i parametri sono dovutamente mediati sui ranges di interesse, che sono 8:00->18:59, 19:00->1:59, e 2:00->7:59

I dati saranno quindi nella forma \
x = ((Parametri giorno i-2),(parametri giorno i-1)) \
y = #tweets giorno i

In [23]:
# Mi calcolo le somme dei tweets delle due distinzioni temporali che voglio assumere
# Questi dati compariranno nelle y
N_giorni = len(counts_x_day)
y_mattina = np.zeros(N_giorni)
y_sera = np.zeros(N_giorni )
for i in range(N_giorni):
    temp = counts_x_day[i]
    y_mattina[i] = temp[linspace(15,37,dtype ='int')].sum() # 8->19
    y_sera[i] = temp[linspace(38,47,dtype ='int')].sum()    # 19->24
        
#Questi dati vanno nelle X nei parametri

sum_tweets = np.zeros(N_giorni)
for i in range(N_giorni):
    sum_tweets[i] = counts_x_day[i].sum()
    

Inizio a lavorare sulle temperature, dato che ho le temperature prese ogni 30 minuti decido di mediare sui due periodi SERA e MATTINA, in modo da aver un dato valutativo medio decente

In [70]:
# Adesso voglio  costruirmi un vettore che contenga le
dfTemp = weather.drop(columns = ['rain','municipal','geometry'])
#divido il DF in mattina e sera
df_mattina = df[7.5 < df['hours']]
df_mattina = df_mattina[df_mattina['hours'] < 19]
df_sera = df[19 < df['hours']]



T_mat = df_mattina.groupby( by=['month','day'])['temperature'].sum() / 22  # 8 -> 19
T_ser = df_mattina.groupby( by=['month','day'])['temperature'].sum() / 10

print(T_mat)
print(T_ser)

#T_ser = pd.DataFrame({'Counts' : df_sera.groupby(
#                    ['month','day','temperature']).size()}).reset_index()
#
#T_mattina = T_mat.groupby('temperature').sum()
#print(T_mattina)
#
#print(T_mat)
#print(T_ser)
#
#for i in range(N_giorni):
#    temp_mattina = 
#    temp_sera
#for i in range(N_giorni):
#    T = group_df['temperature'].
#print(T)
#T_mattina = np.zeros(N_giorni)
#T_sera = np.zeros(N_giorni)
#for i in range(N_giorni):
#    temp =     # scelta infelice di chiamare 
#    T_mattina[i] = temp[linspace(15,37,dtype ='int')].sum() # 8->19
#    T_sera[i] = temp[linspace(38,47,dtype ='int')].sum()    # 19->24
#

month  day
11     1      89.447826
       2      83.208696
       3      64.100000
       4      32.669565
       5      52.952174
                ...    
12     27     64.673913
       28     25.143478
       29     73.013043
       30     -6.830435
       31    -30.086957
Name: temperature, Length: 61, dtype: float64
month  day
11     1      205.73
       2      191.38
       3      147.43
       4       75.14
       5      121.79
               ...  
12     27     148.75
       28      57.83
       29     167.93
       30     -15.71
       31     -69.20
Name: temperature, Length: 61, dtype: float64


In [ ]:
columns = ['#tweetsMAT-2','#tweetsMAT-1','#tweetsSER-2','#tweetsSER-1',  ]
ML_df = pd.DataFrame(columns =)


In [103]:
type(linspace(16,38, 23)[4])

numpy.float64

In [100]:
r = counts_x_day[16]
r[linspace(16,38, 23, dtype ='int')]

array([ 1,  3,  7,  6,  4,  3,  3,  9, 12,  2,  7,  8,  6, 12,  5, 13, 11,
        7,  7,  6,  5,  7, 11], dtype=int64)

## Logistic Regressor